In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [2]:
!pip install U "langchain[google-genai]"

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 630.8 kB/s eta 0:00:02
   --------------- ------------------------ 0.5/1.4 MB 630.8 kB/s eta 0:00:02
   ---------------------- ----------------- 0.8/1.4 MB 628.9 kB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 628.9 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 630.4 kB/s eta 0:00:01
   -------------------------------------- - 1.3/1.4 MB 634.3 kB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 619.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -------------------------------------

In [3]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [3]:
llm.invoke("Who is the PM of India")

AIMessage(content='The current Prime Minister of India is Narendra Modi.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--3f1950af-2e27-4e87-bc94-f0cfaea0b45d-0', usage_metadata={'input_tokens': 6, 'output_tokens': 11, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})

In [4]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
!pip install U langchain-chroma

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached huggingface_hub-0.32.3-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
    --------------------------------------- 0.3/19.3 MB ? eta -:--:--
    --------------------------------------- 0.3/19.3 MB ? eta -:--:--
    --------------------------------------- 0.3/19.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.3 MB 349.4 kB/s eta 0:00:54
   - ------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.73.0rc1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.


In [6]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="basic_rag",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [7]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [10]:
len(all_splits)

63

In [11]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [12]:

prompt = hub.pull("rlm/rag-prompt")

c:\Users\vijay\miniconda3\envs\vijay\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [14]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [15]:
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

In [16]:
def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [17]:
l = ["Vijay","Hemanth","Tagore"]

In [18]:
l

['Vijay', 'Hemanth', 'Tagore']

In [19]:
print("".join(l))

VijayHemanthTagore


In [20]:
print(" ".join(l))

Vijay Hemanth Tagore


In [21]:
print(" &".join(l))

Vijay &Hemanth &Tagore


In [22]:
print("\n".join(l))

Vijay
Hemanth
Tagore


In [23]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [24]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is breaking down a complex task into smaller, more manageable steps. This can be achieved through prompting the model to "think step by step" or by using task-specific instructions. Task decomposition can also be done with human inputs.


In [25]:
response = graph.invoke({"question": "who is the PM of India"})
print(response["answer"])


I'm sorry, but the provided text does not contain information about who the Prime Minister of India is. The context discusses the use of language models and APIs in different applications. Therefore, I cannot answer your question.


In [26]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])